In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


# SciBERT 

## Load ZO_UP data

In [3]:
%run ../ipynb_load_data_natural.py

{'SDG': Value(dtype='int64', id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}
Example instance:	 {'SDG': 8, 'ABSTRACT': 'The scheme gives enterprises with business activity in Norway a tax credit on their R&D projects. The R&D content must be approved by the Research Council of Norway ex ante. In 2009, the cap on expenses per enterprise for intramural R&D projects increased to NOK 5.5 million (previously it was N0K 4 million), and NOK11 million (previously it was NOK 8 million) for projects conducted at an R&D institution.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None, '__index_level_0__': 492}
id2label: {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10', 10: '11', 11: '12', 12: '13', 13: '14', 14: '15', 15: '16', 16: '17'}
label2id: {'1': 0, '2': 1, '

## Model

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

HF_MODEL_NAME = "allenai/scibert_scivocab_cased"
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    id2label=id2label,
    label2id=label2id
).to("cuda")

encoded_dataset = dataset.map(
    preprocess_data(tokenizer), batched=True, remove_columns=dataset["train"].column_names
)
encoded_dataset.set_format("torch")

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references

Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 8
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/final/{HF_MODEL_NAME}-{DATASET_TYPE.value}",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
    run_name=f"{HF_MODEL_NAME}-{DATASET_TYPE.value}"
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.838700,2.758859,0.103321,0.104502,0.103321,0.077099
2,2.668700,2.611231,0.258303,0.284932,0.258303,0.228446
3,2.439800,2.360955,0.435424,0.406208,0.435424,0.381253
4,2.122300,2.063442,0.575646,0.585942,0.575646,0.545970
5,1.824700,1.816852,0.653137,0.668255,0.653137,0.625459
6,1.566500,1.590736,0.708487,0.698384,0.708487,0.690903
7,1.340500,1.422202,0.726937,0.727355,0.726937,0.711811
8,1.140100,1.277674,0.752768,0.754191,0.752768,0.744618
9,0.972600,1.191291,0.738007,0.739331,0.738007,0.728387
10,0.839300,1.113578,0.756458,0.756173,0.756458,0.747698


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/m

TrainOutput(global_step=540, training_loss=1.125635016406024, metrics={'train_runtime': 370.1015, 'train_samples_per_second': 34.045, 'train_steps_per_second': 1.459, 'total_flos': 3315645785088000.0, 'train_loss': 1.125635016406024, 'epoch': 20.0})

In [7]:
trainer.evaluate()

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.9529668688774109,
 'eval_accuracy': 0.7712177121771218,
 'eval_precision': 0.7719573695224307,
 'eval_recall': 0.7712177121771218,
 'eval_f1': 0.767099786720493,
 'eval_runtime': 1.3291,
 'eval_samples_per_second': 203.902,
 'eval_steps_per_second': 9.029,
 'epoch': 20.0}

In [8]:
from sklearn.metrics import classification_report

model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))], digits=4)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [9]:
idx = 4
print(encoded_dataset["train"][idx]["label"])

# decode the token input_ids to text
tokenizer.decode(encoded_dataset["train"][idx]["input_ids"])

tensor(6)


'[CLS] trends in the phase stability and thermochemical oxygen exchange of ceria doped with potentially tetravalent metals ceria is among the most prominent materials for generating clean fuels through solar thermochemical co2 reduction and water splitting. the main optimization parameter for ceria in solar reactors is the oxygen exchange capacity ( oec, dd ), which can be notably improved through various dopant types. among them, tetravalent dopants excel through the formation of active vacancies which lead to particularly high oec values. we thus performed a comprehensive screening evaluation of all dopants in the periodic table which have been reported to adopt an oxidation state of + iv. all thermally stable doped ceria samples, m0. 1ce0. 9o2d ( m [UNK] si, ti, v, cr, zr, nb, rh, hf, ta, nb, v, pr, and tb ), were first analyzed for dd improvement with thermogravimetric analysis ( tga ). dopant solubility limits and behavior in the ceria host lattice was evaluated with scanning elec

In [10]:
print(report)

              precision    recall  f1-score   support

       SDG 1     0.8750    0.8235    0.8485        17
       SDG 2     0.8000    0.9412    0.8649        17
       SDG 3     0.8235    0.8235    0.8235        17
       SDG 4     0.9444    1.0000    0.9714        17
       SDG 5     0.7500    0.7059    0.7273        17
       SDG 6     0.8000    1.0000    0.8889        16
       SDG 7     0.6316    0.7500    0.6857        16
       SDG 8     0.6875    0.6471    0.6667        17
       SDG 9     0.6667    0.7059    0.6857        17
      SDG 10     0.5625    0.5294    0.5455        17
      SDG 11     0.9375    0.8824    0.9091        17
      SDG 12     0.7273    0.4706    0.5714        17
      SDG 13     0.8667    0.7647    0.8125        17
      SDG 14     0.8889    0.9412    0.9143        17
      SDG 15     0.8571    0.7059    0.7742        17
      SDG 16     0.5714    0.7059    0.6316        17
      SDG 17     0.0000    0.0000    0.0000         1

    accuracy              

In [11]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
import transformers
pred = transformers.pipeline(
    "text-classification",
    model=model,
    batch_size=8,
    tokenizer=tokenizer,
    truncation=True,
    padding=True,
    max_length=512,
    device=0,
    top_k=None,     # equal to return_all_scores=True
)

sample_sentence = """Ensure access to affordable, reliable, sustainable and modern energy for all """
pred([sample_sentence])

[[{'label': '7', 'score': 0.6354700922966003},
  {'label': '12', 'score': 0.08636032044887543},
  {'label': '9', 'score': 0.052231889218091965},
  {'label': '13', 'score': 0.04780197888612747},
  {'label': '6', 'score': 0.024476656690239906},
  {'label': '3', 'score': 0.01861574500799179},
  {'label': '10', 'score': 0.017527032643556595},
  {'label': '4', 'score': 0.016823742538690567},
  {'label': '14', 'score': 0.016308670863509178},
  {'label': '1', 'score': 0.01359023991972208},
  {'label': '8', 'score': 0.012797074392437935},
  {'label': '11', 'score': 0.011187211610376835},
  {'label': '2', 'score': 0.010784538462758064},
  {'label': '5', 'score': 0.009997137822210789},
  {'label': '16', 'score': 0.009833763353526592},
  {'label': '15', 'score': 0.008207985199987888},
  {'label': '17', 'score': 0.007985986769199371}]]

In [13]:
probs = torch.nn.functional.softmax(model(**tokenizer(sample_sentence, return_tensors="pt", truncation=True, max_length=512).to(model.device)).logits, dim=-1)

for i, prob in enumerate(probs[0]):
    print(f"Label {model.config.id2label[i]}: {prob.item()}")

Label 1: 0.013590238988399506
Label 2: 0.01078453753143549
Label 3: 0.018615741282701492
Label 4: 0.016823740676045418
Label 5: 0.00999713595956564
Label 6: 0.024476652964949608
Label 7: 0.6354699730873108
Label 8: 0.012797071598470211
Label 9: 0.05223188176751137
Label 10: 0.017527032643556595
Label 11: 0.01118721067905426
Label 12: 0.08636032044887543
Label 13: 0.04780197888612747
Label 14: 0.016308670863509178
Label 15: 0.008207984268665314
Label 16: 0.009833762422204018
Label 17: 0.007985986769199371
